In [12]:
import rospy
import json
import glob
import rospkg
from os.path import join
from numpy import mean
from scipy.stats import sem
import pylab as plt
from os.path import basename

In [2]:
def plot_analysis(resfiles, resdir):
        width = 0.2

        fig = plt.figure(facecolor="white")
        ax = fig.add_subplot(111)
        plt.rcParams['font.size'] = 20
        
        labels = ['fixed', 'position only', 'orientation only', 'position \nand orientation']
        colors = ['tomato', 'darkseagreen', 'cornflowerblue', 'slategray']
        conditions = ['fixed', 'orientation_only', 'pose', 'position_only']
        score_dict = {}
        for c in conditions:
            score_dict[c] = []
        
        # plot figure for each result file
        for f in resfiles:
            with open(f) as data_file:
                data = json.load(data_file)
            for c in conditions:
                score_dict[c] += data[c]

        for i in range(len(conditions)):
            score = score_dict[conditions[i]]
            ax.bar((i + 1) / 2.0 - width / 2, mean(score),
                   width, color=colors[i], label=labels[i])
            (line, caps, _) = plt.errorbar((i + 1) / 2.0, mean(score),
                                           sem(score), capsize=3, elinewidth=2, ecolor='k')

            line.remove()
            for cap in caps:
                cap.set_color('k')
                cap.set_markeredgewidth(3)

        legend = ax.legend(loc='upper right')
        # Set the fontsize
        for label in legend.get_texts():
            label.set_fontsize('small')

        for label in legend.get_lines():
            label.set_linewidth(1.5)  # the legend line width

        ax.set_ylabel('Average REBA Score')
        ax.set_ylim([0, 8])
        ax.set_xlim([0, 2.5])

        # save svg file
        savefile = join(resdir, 'average_reba_cost')
        fig.set_size_inches(12.8, 10.24)
        plt.savefig(savefile + '.svg', dpi=100, facecolor=fig.get_facecolor(), transparent=False)
        plt.close()

In [51]:
def find_best_config(resdir):
    resfiles = glob.glob(join(resdir, '*'))
    best_score = 100
    best_id = 0
    for f in resfiles:
        with open(join(f, 'cost_details.json')) as datafile:
            data = json.load(datafile)
        score = max(data['pose']['reba'])
        score = data['pose']['mean_reba']
        if  score < best_score:
            best_id = basename(f)
            best_score = score
    return best_id, best_score

In [5]:
rospy.init_node('welding_analyzer')

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [58]:
res_folder = 'test'

In [59]:
rospack = rospkg.RosPack()
pkg_expe = rospack.get_path('thr_xp_human_comfort')
directory = join(pkg_expe, 'results', res_folder)

In [66]:
find_best_config(directory)

('6324', 3.447674870491028)

In [37]:
directory

'/home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/welding_simulation_test2'